# Simulate Raman Spectra using **Pyscf** package

In [1]:

from pyscf import gto
from pyscf.hessian import thermo
import numpy as np
import py3Dmol
from pyscf.geomopt.geometric_solver import optimize
import pandas as pd
import plotly.express as px
from pyscf.prop.polarizability.rhf import Polarizability



/home/bubbleinternal/miniconda3/envs/EnvForPyscf/lib/python3.8/site-packages/pyscf/prop/efg/rhf.py:45: UserWarning: Module EFG is under testing
  warnings.warn('Module EFG is under testing')
/home/bubbleinternal/miniconda3/envs/EnvForPyscf/lib/python3.8/site-packages/pyscf/prop/efg/dhf.py:30: UserWarning: Module EFG is under testing
  warnings.warn('Module EFG is under testing')
/home/bubbleinternal/miniconda3/envs/EnvForPyscf/lib/python3.8/site-packages/pyscf/prop/zfs/uhf.py:40: UserWarning: Module ZFS is under testing
  warnings.warn('Module ZFS is under testing')
/home/bubbleinternal/miniconda3/envs/EnvForPyscf/lib/python3.8/site-packages/pyscf/prop/gtensor/uhf.py:43: UserWarning: Module g-tensor is under testing
  warnings.warn('Module g-tensor is under testing')
/home/bubbleinternal/miniconda3/envs/EnvForPyscf/lib/python3.8/site-packages/pyscf/prop/gtensor/uks.py:41: UserWarning: Module g-tensor is under testing
  warnings.warn('Module g-tensor is under testing')
/home/bubbleinter

# Raman Scattering
Raman spectroscopy is a widely used technique in chemistry and biology to figure out molecular composition and structure of samples by measuring the vibrational energy states. To see this, we need to go through some of the basic theory for Raman spectroscopy. 


<img src = "../figures/Raman-Scattering-1.png" alt = "Raman Scattering" width="600">

<img src = "../figures/Jablonski-Diagram-1.png" alt = "Raman Scattering" width="600">

*edinst.com/blog/what-is-raman-spectroscopy/*

In general, when you shoot a beam of light onto some molecules, you would pump the molecule from one of the ground vibrational state to the virtual state, which is created by the interaction between the light and the molecule. Since virtual state is not stable, the molecule will come down again to the ground vibrational states. Three types of event tends to happens in this process. The first one is what people called **Rayleigh** scattering, where the molecule drop from the virutal state to the original vibrational ground state, causing no change in the scattered photon energy comparing to the incident photon.

The other two types of event involves change of scattered photon energy due to the fact that the molecule drop from virtual state to a different vibrational ground states. Depends on whether it drop to a higher or lower ground state compare to the original, people call them **Stokes** or **Anti-stokes** Raman scattering. And the shifts in the photon energy tells us information about the vibrational states of the molecules, which serves as a ***finger print*** due the uniqueness of vibrational frequencies for different molecules. 

We can understand the Raman process in term of classcial theory. When a molecule is in an electric field $\mathbf{E}$, the molecule will be polarized, which result with an induced dipple moment $\mathbf{P}$. The strength of the dipole $\mathbf{P}$ is given by
$$
\mathbf{P} = \alpha E
$$
where $\alpha$ is the polarizability of the molecule.
Light is a electromagnetic wave. For a beam of light with frequency $\nu_0$, the magntiude of the electric field can be written as
$$
\mathbf{E} = E_0 \cos (2 \pi \nu_0 t).
$$ 
As a result, the induced dipole of the molecule can be derived:
$$
\mathbf{P} = \alpha \mathbf{E_0} \cos (2 \pi \nu_0 t)
$$
The polarizability will depends on the actual geometry of the molecule. So as you can image, when the molecule starts to vibrate, the polarizbaility will change. We can write polarizability $\alpha$ as a function of vibrational modes $\mathbf{q}$. If we do a first order taylor expansion of $\alpha$, then
$$
\alpha = \alpha_0 + (\frac{\partial \alpha}{\partial \mathbf{q_i}})\cos (2 \pi \nu_i t),
$$
where $\nu_i$ is the $i$ th vibrational frequency respect to the corresponding modes. Plug it back in to the induce dipole equation
$$
\mathbf{P} = \alpha_0 \mathbf{E_0} \cos (2 \pi \nu_0 t) + (\frac{\partial \alpha}{\partial \mathbf{q_i}})\mathbf{E_0} \cos (2 \pi \nu_0 t)\cos (2 \pi \nu_i t) \\

= \alpha_0 \mathbf{E_0} \cos(2 \pi \nu_0 t) + {1\over 2}(\frac{\partial \alpha}{\partial \mathbf{q_i}})\mathbf{E_0} [\cos (2 \pi (\nu_0 - \nu_i) t) + \cos (2 \pi (\nu_0 + \nu_i ) t)].
$$ 

The first term represent the Rayleigh scattering and the last two term represent anti-stokes and stokes scattering process. Since the scattering intensity is proportional to $|\mathbf{P}|^2$, the Raman intensity $I_{Raman}$ is proportional to $\large |\frac{\partial \alpha}{\partial \mathbf{q_i}}|^2$. 

## Procedure to calculate Raman Spectrum
Here we will present a native implementation of simulating static Raman spectrum for the given molecule using ***Hartree Fock*** method from ```Pyscf``` package.

The first step for simulating Raman scattering is to construct the molecule we would like to study. For ```Pyscf``` package, we need to feed the xyz position of the atoms and the basis function for constructing the wave functon. Here I showed an example for H2O molecule.

In [ ]:
# Construct water molecule. The position are default in unit of angstrom.
mol = gto.M(
    atom="""
    O  0.000000  0.000000  0.000000
    H  0.758602  0.000000  0.504284
    H -0.758602  0.000000  0.504284
    """,
    basis='6-31g', # I used one of the simpliest basis set
    verbose=0, # Cancel most output information
)

# 3D view of the molecule
print("Molecule before optimize")
xyz_view_before = py3Dmol.view(width=400,height=400)
xyz_view_before.addModel(mol.tostring(format="xyz"),'xyz')
xyz_view_before.setStyle({'stick':{}, "sphere":{"radius":0.4}})
xyz_view_before.setBackgroundColor('0xeeeeee')
xyz_view_before.show()


Molecule before optimize


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In general, for different basis function, the equilibrium structure will be slightly different. Thus, for later calculation, it's a good idea to obtain the correct equilibrium structure. We will be using the optimizer provided by ```Pyscf```. In the water case, it does not care that much. But for much bigger molecule such as glycine, it will make a noticable difference, as we will see

In [3]:

# Set up restricted hartree fork object
mf = mol.RHF().run()

# Excute geometric optimization of the molecule using pyscf geometric solver
Mol_freq = optimize(mf)

# Update atomic position using optimized result
mol.set_geom_(Mol_freq.tostring())

# 3D view of the optimized molecule
print("Molecule after optimization")
xyz_view_after = py3Dmol.view(width=400,height=400)
xyz_view_after.addModel(mol.tostring(format="xyz"),'xyz')
xyz_view_after.setStyle({'stick':{}, "sphere":{"radius":0.4}})
xyz_view_after.setBackgroundColor('0xeeeeee')
xyz_view_after.show()

geometric-optimize called with the following command line:
/home/bubbleinternal/miniconda3/envs/EnvForPyscf/lib/python3.8/site-packages/ipykernel_launcher.py --f=/home/bubbleinternal/.local/share/jupyter/runtime/kernel-v3128d00e1b2772f54a4444d3b8f2e42a9881ad30d.json

                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
            

Molecule after optimization


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

After we set up the stage for accurate computation, we need three piece of information for simulating Raman spectrum: the vibrational frequencies, the normal modes and the polarizability derivative respect to the normal modes. The vibratonal frequencies and normal modes can be obtained by calculating the **Hessian Matrix** of the molecule, which is defined to be
$$
M_{H}^{ij} = \frac{\partial ^2 E}{\partial \mathbf{x_i} \partial \mathbf{x_j} }
$$  
where $\mathbf{x_i}$ represent the $i$ th atomic coordinate. The eigenvalue and eigen vector of this matrix will give us the information about the vibrational frequencies and normal mode. Here we used the built-in function *harmonic_analysis* from ```pyscf.thermo``` module. 

In [ ]:
# Run unrestricted hatree fock method
mf=mol.RHF().run()

# Compute Hessian matrix for normal mode calculation
Hessian_M = mf.Hessian().kernel()

# Calculate normal modes and frequencies using harmonic_analysis
freq_info = thermo.harmonic_analysis(mf.mol,Hessian_M,exclude_rot=True,imaginary_freq=True)


# Assign frequencies and normal mode for future use
Vib_freq = np.transpose(freq_info["freq_wavenumber"])
Nrm_mde = freq_info["norm_mode"]



Now we know the normal modes of the molecule, we can compute the derivative of polarizability respect to the normal modes. We will use finite differences method to compute the numerical derivative
$$
\frac{\partial \alpha_{ij}}{\partial q_k} \approx {\alpha(\mathbf{x}+\delta q_k)-\alpha(\mathbf{x}-\delta q_k) \over 2 \delta q_i}.
$$
Then the corresponding Raman intensity can be approximate by 
$$
I_i \propto |\frac{\partial \alpha_{ij}}{\partial q_k}|^2
$$

In [5]:

displacement = 0.0005  # Small displacement in Angstrom

polarizability_tensors = np.zeros((np.size(Vib_freq),3,3))

def compute_polarizability(mol, displacement, Normal_modes, mode_idx, sign):
    """
    Compute the polarizability tensor after displacing along a normal mode.
    """
    new_coords = mol.atom_coords()
    displacements = Normal_modes[mode_idx]* displacement * sign
    new_coords += displacements
    mol_c = mol.copy()
    mol_c.set_geom_(new_coords, unit='ANG')
    displaced_mf = mol.RHF().run()
    polar_tensor = Polarizability(displaced_mf).polarizability_with_freq(freq=0)
    return np.array(polar_tensor)



for mode_idx in range(np.size(Vib_freq)):
    forward_tensor = compute_polarizability(mol, displacement,Nrm_mde, mode_idx, +1)
    backward_tensor = compute_polarizability(mol, displacement, Nrm_mde, mode_idx, -1)
    # Compute finite difference derivative
    polarizability_tensors[mode_idx] = (forward_tensor - backward_tensor) / (2 * displacement)


raman_intensities = np.zeros(np.size(Vib_freq))

for mode_idx, tensor in enumerate(polarizability_tensors):
    raman_intensities[mode_idx]=(np.einsum('ij->',tensor))**2

def gaussian(x, mu, sig=30):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))


x_range = np.linspace(Vib_freq.min()*0.5, Vib_freq.max()*1.1, num=1000)

intensity = np.zeros(x_range.size)

for e, f in zip(Vib_freq, raman_intensities):
    intensity += gaussian(x_range, e) * f
    
# Rough normalization of intensity
dx = (x_range[-1] - x_range[0])/x_range.size
area = (intensity*dx).sum()
intensity /= area

# Plot raman intensity versus wave number
data = pd.DataFrame({"wave number (cm^-1)":x_range, "Intensity (au)":intensity})
fig = px.line(data, x="wave number (cm^-1)", y="Intensity (au)", markers=True)
fig.show()

In [ ]:
# moldesc =   'H 2.0471 0.1029 0.7251;C 1.4376 0.0818 -0.1875;H 1.6643 0.9894 -0.7620;H 1.7616 -0.7796 -0.7860;C -0.0377 -0.0035 0.0968;O -0.4797 -0.0446 1.2294;C -0.9294 -0.0345 -1.1150;H     -0.7844      0.8703     -1.7195;H     -0.6856     -0.8986     -1.7467;H     -1.9948     -0.0979     -0.8587'
# mol = gto.Mole(atom=moldesc, basis='ccpvdz',verbose=0)
# mol.build()


# Define the glycine molecule with its atomic xyz coordinate in angstrom
# atom_c = 'O  1.081302  1.129735  1.195158; O  -.967942  1.693585   .543683; N  2.060859  1.075277 -1.315237;C   .249391  1.494424   .336070; C   .760991  1.733681 -1.081882; H  2.396597  1.201189 -2.305828; H  2.790965  1.427758  -.669398; H  1.985133   .067145 -1.148141; H   .883860  2.805965 -1.234913; H   .041439  1.369111 -1.813528'
# mol = gto.Mole(atom = atom_c)

# # Choose the basis set wave function for initial guess. 
# # I used Sadlej basis for better electric properties calculation
# mol.basis = "ccpvdz"

# # Choose the information output level by the algorithm
# mol.verbose=0

# # Generate molecule object
# mol.build()